<a href="https://colab.research.google.com/github/laicheil/force2019/blob/master/colab/tf_keras_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install  --upgrade laicheil.force2019==0.post0.dev6
from laicheil.force2019 import something
something()

In [0]:
from tensorflow.python.client import device_lib
import numpy as np
device_lib.list_local_devices()

Upload the data files

In [0]:
from google.colab import files

uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))

Saving hackathon_training_data.zip to hackathon_training_data.zip


In [0]:
!mkdir hackathon_training_data 
!unzip hackathon_training_data.zip -d hackathon_training_data
!ls hackathon_training_data/

In [0]:
import os
import json
data_path = 'hackathon_training_data'
list_of_files = os.listdir(data_path)
num_of_files = len(list_of_files)
first_file_path = os.path.join(data_path, list_of_files[0])
#print (first_file_path)
with open(first_file_path,'r') as read_file:
  shape_of_files = (num_of_files,) + np.asarray(json.load(read_file)).shape + (1, )
#print (shape_of_files)
data = np.zeros((shape_of_files))
labels = np.zeros(num_of_files)
labels_ce = np.zeros((num_of_files,2))
for i, filename in enumerate(os.listdir(data_path)):
    full_path = os.path.join(data_path,filename)
    labels[i] = int(filename.startswith('good'))
    labels_ce[i, int(filename.startswith('good'))] = 1
    with open(full_path,'r') as read_file:
        data[i, :, :, 0] = np.asarray(json.load(read_file))

print('labels shape', labels.shape)
print('labels for CE shape', labels_ce.shape)
print('data shape', data.shape)

labels shape (200,)
labels for CE shape (200, 2)
data shape (200, 40, 39, 1)


In [0]:
print(labels)
print(os.listdir(data_path))



Image data generators for the inputs

In [0]:
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

datagen = image.ImageDataGenerator (
    featurewise_center = True,
    featurewise_std_normalization=True,
    vertical_flip=True,
    horizontal_flip=True,
    rotation_range=90)
datagen.fit (data)

train_samples, validation_samples, train_labels, validation_labels = train_test_split(data, labels, test_size=.334)

train_generator         = datagen.flow(train_samples, train_labels, batch_size=32)
validation_generator    = datagen.flow(validation_samples , validation_labels , batch_size=32)


train_samples_ce, validation_samples_ce, train_labels_ce, validation_labels_ce = train_test_split(data, labels_ce, test_size=.334)
train_ce_generator         = datagen.flow(train_samples_ce, train_labels_ce, batch_size=32)
validation_ce_generator    = datagen.flow(validation_samples_ce , validation_labels_ce , batch_size=32)

test_ce_generator = validation_ce_generator

[link text](https://)Loading the ResNet50 model from the tensorflow-keras library

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.33

from tensorflow.python.keras import backend as K
#with tf.device('/device:GPU:0'):
K.set_session (tf.Session (config = config))

print('DONE LOADING MODEL')

DONE LOADING MODEL


Callbacks

In [0]:

import datetime

now = datetime.datetime.now ()
date_str = now.strftime('%Y%m%d%H%M')
checkpoint_init_name = 'init_chkpnt_'+date_str+'.hdf5'
from tensorflow.python.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint
callbacks = [ 
        EarlyStopping (monitor='val_acc', patience=9, verbose=1),
        ModelCheckpoint(checkpoint_init_name, monitor='val_acc', save_best_only=True, save_weights_only=True, verbose=1)
        ]

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Lambda, Dense, Flatten
from tensorflow.image import grayscale_to_rgb

## inputs
inputs = Input (shape=data.shape[1:])#samples.shape[1:]
#
## from grayscale to RGB, Xception needs 3 Channel input
x = Lambda (lambda x: grayscale_to_rgb (x), name='grayscale_to_rgb') (inputs) 
base_model = ResNet50(weights='imagenet', input_tensor=x,include_top=False)
output = Flatten()(base_model.output)
output = Dense(1000, activation='relu')(output)
output = Dense(100, activation='relu')(output)
output = Dense(2, activation='softmax')(output)
## The model
num_layers = len(base_model.layers)
#for i, layer in enumerate (base_model.layers):
#  layer.trainable = i < 8 or i > num_layers-8
model = Model (inputs=inputs, outputs=output)
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
print(model.output_shape)

(None, 2)


Train

In [0]:
model.fit_generator(train_ce_generator, steps_per_epoch=int(train_samples.shape[0]), epochs=100,validation_data=validation_ce_generator)
evaluation = model.evaluate_generator(validation_ce_generator)

print(evaluation)

Train using kfold

In [0]:
from sklearn.model_selection import KFold

k_checkpoint_basename = 'CHK_' + date_str + '_K'
kf = KFold (shuffle=True, n_splits=5)
last_good_model_weights = ''
k=0
for train_index, test_index in kf.split(data, labels_ce):
  print('At fold K=',k,' with ', len(train_index), ' samples out of total ', data.shape[0])
  kf_filepath=k_checkpoint_basename + str(k) + '.hdf5'
  callbacks[-1].filepath = kf_filepath
  history = model.fit_generator (generator       = datagen.flow(data[train_index], labels_ce[train_index], batch_size=16), 
                                 validation_data = datagen.flow(data[test_index] , labels_ce[test_index] , batch_size=16),
                                 steps_per_epoch = int(data.shape[0]/4), 
                                 epochs          = 1, 
                                 callbacks       = callbacks)
  if os.path.isfile(kf_filepath):
    #model.load_weights (kf_filepath) #Load best
    last_good_model_weights = kf_filepath
  if os.path.isfile(last_good_model_weights):
    model.load_weights (last_good_model_weights)
  evaluation = model.evaluate_generator(test_ce_generator)
  print ('Evaluation Mean Squared Error on test data for k =', k, 'is:', evaluation*100.)
  folds_map [k] = {
      'evaluation'   : evaluation,
      'history'      : history,
      'filepath'     : kf_filepath } 
  k += 1


In [0]:

evaluation = model.evaluate_generator(validation_ce_generator)
predict = model.predict_generator(validation_ce_generator)

print(evaluation)
print(predict)